In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install ipywidgets --quiet


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys

sys.path.append("..")

In [4]:
import ipywidgets as widgets
import torch

from discrete_flow_matching.model import DiscreteFlowMatchingNet

In [5]:
model = DiscreteFlowMatchingNet.load_from_checkpoint(
    "../flow-matching-tiny-stories/1pkkmee9/checkpoints/epoch=0-step=3000.ckpt"
)
model.freeze()
model.eval()
model.to(dtype=torch.bfloat16, device="cuda:0")
model

DiscreteFlowMatchingNet(
  (input_projection): Embedding(50259, 768)
  (embed_timestep): Sequential(
    (0): Embedding(1024, 768)
    (1): Unsqueeze()
  )
  (blocks): ModuleList(
    (0-5): 6 x Sequential(
      (0): Transpose()
      (1): Conv1d(768, 768, kernel_size=(31,), stride=(1,), padding=same)
      (2): Transpose()
      (3): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=768, out_features=768, bias=True)
      (6): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (7): GELU(approximate='none')
    )
  )
  (timestep_embedding_norms): ModuleList(
    (0-5): 6 x LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (output_projection): Linear(in_features=768, out_features=50259, bias=True)
)

In [54]:
step_tokens: list[list[str]] = []
step_texts: list[str] = []
with torch.inference_mode():
    for t, samples in model.sample(
        num_samples=1,
        sequence_length=128,
        num_sampling_steps=32,
        stochasticity=5.0,
        yield_intermediate=True,
    ):
        step_tokens.append(model.tokenizer.batch_decode(samples[0]))
        step_texts.append(model.tokenizer.decode(samples[0]))

In [55]:
def get_step_text(step):
    rows = []
    column_size = 16
    for i in range(0, len(step_tokens[step]), column_size):
        rows.append(step_tokens[step][i : i + column_size])

    # Create html table
    html_table = '<table style="border: 1px solid black">'
    for row in rows:
        html_table += "<tr>"
        for token in row:
            style = "border: 1px solid black; width: 50px; text-align: center;"
            if token == "[MASK]":
                style += "background-color: #cccccc;"
            else:
                style += "background-color: #eeeeee;"
            html_table += f'<td style="{style}">{token}</td>'
        html_table += "</tr>"
    html_table += "</table>"

    return widgets.HTML(html_table)


interact_widget = widgets.interact(get_step_text, step=(0, len(step_tokens) - 1, 1))


interactive(children=(IntSlider(value=16, description='step', max=32), Output()), _dom_classes=('widget-intera…